Binary heterochromatin classification
- Two categories 
    1. either chromatin state 13 
    2. or one of the other 25 categories


In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, LSTM
from keras.layers import Lambda, Convolution1D, MaxPooling1D, Flatten, Reshape
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.metrics import categorical_crossentropy, binary_crossentropy
#For data saving
import pickle
import random
#other imports
import gzip
import glob
import os
import keras.backend as K
import os
#cwd = os.path.dirname(os.path.realpath("SURF_001_TwoClass13.ipynb"))

### Loading algorithims and labels

In [ ]:
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY']

In [ ]:
def oneHot_DNA(chrom):
    """
    Given a chromosome array with a, t, c, and g's will output a one-hot encoded vector array
    @return one-hot encoded numpy array
    """
    one_hot_full = np.zeros((len(chrom), len(chrom[0]), 4), dtype=np.int8)
    for i, seq in enumerate(chrom):
        seq_onehot = np.zeros((len(seq), 4))
        for j, nuc in enumerate(seq):
            if nuc == 'a':
                seq_onehot[j, :] = np.array([1, 0, 0, 0], dtype=np.int8)
            elif nuc == 't':
                seq_onehot[j, :] = np.array([0, 1, 0, 0], dtype=np.int8)
            elif nuc == 'c':
                seq_onehot[j, :] = np.array([0, 0, 1, 0], dtype=np.int8)
            elif nuc == 'g':
                seq_onehot[j, :] = np.array([0, 0, 0, 1], dtype=np.int8)
            one_hot_full[i,:,:] = seq_onehot
    return one_hot_full

In [ ]:
def oneHot_DNA_full_PaddingZeros(chrom, max):
    """ Given a chromosome array with sequences. Will pad sequeneces to max length given and will shorten any sequences longer than max
    
    @return one-hot encoded numpy array with Zero-padding
    """
    largest_size = max
    #for seq in chrom:
    #  size = len(seq)
    #  if size > largest_size:
    #    largest_size = size
    one_hot_full = np.zeros((len(chrom), largest_size, 4), dtype=np.int8)
    for i, seq in enumerate(chrom):
        seq_onehot = np.zeros((largest_size, 4))
        for j, nuc in enumerate(seq):
            if nuc == 'a':
                seq_onehot[j, :] = np.array([1, 0, 0, 0], dtype=np.int8)
            elif nuc == 't':
                seq_onehot[j, :] = np.array([0, 1, 0, 0], dtype=np.int8)
            elif nuc == 'c':
                seq_onehot[j, :] = np.array([0, 0, 1, 0], dtype=np.int8)
            elif nuc == 'g':
                seq_onehot[j, :] = np.array([0, 0, 0, 1], dtype=np.int8)
            else:
                print('issue')
            one_hot_full[i,:,:] = seq_onehot
    return one_hot_full

### Onehot encoding the samples and labels:

In [ ]:
genome = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

max_num = 0
for x, indexes in enumerate(genome):
    with open('../DataSet_13_notonehot/DataSet_13_test_samples_1-Version_' + genome[x] + '.dat', 'rb') as f2:
        test_samples = pickle.load(f2)
    with open('../DataSet_13_notonehot/DataSet_13_train_samples_1-Version_' + genome[x] + '.dat', 'rb') as f2:
        train_samples = pickle.load(f2)
    with open('../DataSet_13_notonehot/DataSet_13_test_labels_1-Version_' + genome[x] + '.dat', 'rb') as f2:
        test_labels = pickle.load(f2)
    with open('../DataSet_13_notonehot/DataSet_13_train_labels_1-Version_' + genome[x] + '.dat', 'rb') as f2:
        train_labels = pickle.load(f2)
    for i, label in enumerate(train_labels):
        if label == 0:
            if max_num < len(train_samples[i]):
                max_num = len(train_samples[i])
    for i, label in enumerate(test_labels):
        if label == 0:
            if max_num < len(test_samples[i]):
                 max_num = len(test_samples[i])
    print(max_num)
    print(max_num)

In [ ]:
largest_seq = max_num

In [ ]:
count_a = 0
count_a_place = 0
count_b = 0
count_b_place = 0
count_c = 0
count_c_place = 0
count = 0

for x, indexes in enumerate(genome):
    train_samples = []
    train_labels = []
    with open('../DataSet_13_notonehot/DataSet_13_train_samples_1-Version_' + genome[x] + '.dat', 'rb') as f1:
        train_samples = pickle.load(f1)
        #print(len(train_samples))
    with open('../DataSet_13_notonehot/DataSet_13_train_labels_1-Version_' + genome[x] + '.dat', 'rb') as f2:
        train_labels = pickle.load(f2)
        #print(len(train_labels))
    for y, index in enumerate(train_samples):
        len_seq = len(index)
        new_train_samples = []
        new_train_labels = []
        index_array = []
        index_array.append(index)
        if count_a_place > 10000:
            count_a_place -= 10000
        if count_b_place > 10000:
            count_b_place -= 10000
        if count_c_place > 10000:
            count_c_place -= 10000
        if len_seq <= 1000:
            if count > 9990 and count < 10000:
                new_train_samples = oneHot_DNA_full_PaddingZeros(index_array, 1000)
                new_train_labels = oneHot_labels2_1batch(train_labels[y])
            if count > 9990 and count < 10000:
                print(new_train_samples.shape)
                print(train_labels[y])
                print(new_train_labels)
            count += 1

### Loading all the Data into one Numpy Array

In [ ]:
with open('../partition_train_val_a.dat', 'rb') as f:
    partition = pickle.load(f)
train_samples = partition['train']
val_samples = partition['validation']
random.shuffle(train_samples)
random.shuffle(val_samples)
labels = partition['labels']
partition['train'] = train_samples
partition['validation'] = val_samples

In [ ]:
print(len(train_samples))
print(len(val_samples))
print(len(labels))
print(len(train_samples + val_samples))

In [ ]:
X_train, Y_train = collate_data(train_samples, labels, '../model_11_data_4/', 1000, 2)

In [ ]:
def collate_data(samples, labels, path, sequence_length, num_categories):
    num_train = len(samples)
    Y_train = np.zeros((num_train, num_categories), dtype=np.int8)
    X_train = np.zeros((num_train, sequence_length, 4), dtype=np.int8)
    count = 0
    for i in range(0, num_train):
        a = samples[count]
        X_train[i, :, :] = np.load(path + samples[count] + '.npy') 
        Y_train[i, :] = labels[a] 
        count += 1
    return X_train, Y_train


In [ ]:
with open('../DataSet15/x_train.npy', 'wb') as f:
    pickle.dump(X_train, f)
with open('../DataSet15/y_train.npy', 'wb') as f:
    pickle.dump(Y_train, f)

In [ ]:
X_val, Y_val = collate_data(val_samples, labels, '../model_11_data_4/', 1000, 2)

In [ ]:
with open('../DataSet15/x_val.npy', 'wb') as f:
    pickle.dump(X_val, f)
    print(X_val)
with open('../DataSet15/y_val.npy', 'wb') as f:
    pickle.dump(Y_val, f)
    print(Y_val)

In [ ]:
with open('../DataSet15/x_train.npy', 'rb') as f:
    X_train = pickle.load(f)
    print(X_train.shape)
with open('../DataSet15/y_train.npy', 'rb') as f:
    Y_train = pickle.load(f)
    print(Y_train.shape)


with open('../DataSet15/x_val.npy', 'rb') as f:
    X_val = pickle.load(f)
    print(X_val.shape)
with open('../DataSet15/y_val.npy', 'rb') as f:
    Y_val = pickle.load(f)
    print(Y_val.shape)

In [ ]:
print(X_train.shape[0])
print(X_val.shape[0])
print(X_val.shape[0] + X_train.shape[0])

In [ ]:
def collate_two_arrays(x_train, y_train, x_val, y_val, sequence_length, num_categories):
    num_train = x_train.shape[0] + x_val.shape[0]
    print(num_train)
    Y_train = np.zeros((num_train, num_categories), dtype=np.int8)
    X_train = np.zeros((num_train, sequence_length, 4), dtype=np.int8)
    for i in range(0, x_train.shape[0]):
        X_train[i, :, :] = x_train[i]
        Y_train[i, :] = y_train[i] 
    for i in range(0, x_val.shape[0]):
        X_train[(i + x_train.shape[0]), :, :] = x_val[i]
        Y_train[(i + x_train.shape[0]), :] = y_val[i] 
    return X_train, Y_train


In [ ]:
X_train, Y_train = collate_two_arrays(X_train, Y_train, X_val, Y_val, 1000, 2)

In [ ]:
shuffler = np.random.permutation(X_train.shape[0])
X_train = X_train[shuffler]
Y_train = Y_train[shuffler]

In [ ]:
with open('../DataSet15/x_train_val.npy', 'wb') as f:
    pickle.dump(X_train, f)
    print(X_train)
with open('../DataSet15/y_train_val.npy', 'wb') as f:
    pickle.dump(Y_train, f)
    print(Y_train)

### Combine Test and TrainVal Data sets, shuffle, and then split into two data sets
- one for training and validation
- second one for testing

In [ ]:
def collate_two_arrays(x_array_1, y_array_1, x_array_2, y_array_2, sequence_length, num_categories):
    num_train = x_array_1.shape[0] + x_array_2.shape[0]
    print(num_train)
    Y_train = np.zeros((num_train, num_categories), dtype=np.int8)
    X_train = np.zeros((num_train, sequence_length, 4), dtype=np.int8)
    for i in range(0,x_array_1.shape[0]):
        X_train[i, :, :] = x_array_1[i]
        Y_train[i, :] = y_array_1[i] 
    # minues 2 instead of one because the last value in the testing array is null for some reason so I am getting rid of it
    for i in range(0, x_array_2.shape[0]):
        X_train[(i + x_array_1.shape[0]), :, :] = x_array_2[i]
        Y_train[(i + x_array_1.shape[0]), :] = y_array_2[i] 
    return X_train, Y_train

In [ ]:
X_all, Y_all = collate_two_arrays(X_train, Y_train, X_test, Y_test, 1000, 2)


In [ ]:
shuffler = np.random.permutation(X_all.shape[0])
X_all = X_all[shuffler]
Y_all = Y_all[shuffler]

In [ ]:
with open('../DataSet15/x_all_TRAIN-VAL-TEST.npy', 'wb') as f:
    pickle.dump(X_all, f)
    print(X_all)
with open('../DataSet15/y_all_TRAIN-VAL-TEST.npy', 'wb') as f:
    pickle.dump(Y_all, f)
    print(Y_all)

In [ ]:
print(X_all.shape)
print(Y_all.shape)

In [ ]:
# Splitting for separating out the training and validation dataset
def split_two_arrays(x_array_1, y_array_1, length, sequence_length, num_categories):
    num_train = length
    print(num_train)
    Y_train = np.zeros((length, num_categories), dtype=np.int8)
    X_train = np.zeros((length, sequence_length, 4), dtype=np.int8)
    for i in range(0, length):
        X_train[i, :, :] = x_array_1[i]
        Y_train[i, :] = y_array_1[i] 
    # minues 2 instead of one because the last value in the testing array is null for some reason so I am getting rid of it
    return X_train, Y_train

In [ ]:
x_train_new, y_train_new = split_two_arrays(X_all, Y_all, 67602, 1000, 2)

In [ ]:
# Splitting for separating out the testing data set
def split_two_arrays_2(x_array_1, y_array_1, length, sequence_length, num_categories):
    num_train = length
    print(num_train)
    Y_train = np.zeros((16382, num_categories), dtype=np.int8)
    X_train = np.zeros((16382, sequence_length, 4), dtype=np.int8)
    for i in range(0, 16382):
        X_train[i, :, :] = x_array_1[i + 67602]
        Y_train[i, :] = y_array_1[i + 67602] 
    # minues 2 instead of one because the last value in the testing array is null for some reason so I am getting rid of it
    return X_train, Y_train

In [ ]:
x_test_new, y_test_new = split_two_arrays_2(X_all, Y_all, 16382, 1000, 2)

In [ ]:
with open('../DataSet15/x_train_allshuffled.npy', 'wb') as f:
    pickle.dump(x_train_new, f)
    print(x_train_new.shape)
    print(x_train_new)
with open('../DataSet15/y_train_allshuffled.npy', 'wb') as f:
    pickle.dump(y_train_new, f)
    print(y_train_new.shape)
    print(y_train_new)


In [ ]:
with open('../DataSet15/x_test_allshuffled.npy', 'wb') as f:
    pickle.dump(x_test_new, f)
    print(x_test_new.shape)
    print(x_test_new)
with open('../DataSet15/y_test_allshuffled.npy', 'wb') as f:
    pickle.dump(y_test_new, f)
    print(y_test_new.shape)
    print(y_test_new)
